# Introducción a LGR

In [ ]:
import control
from utils import closed_loop, PID
import numpy as np
from matplotlib.markers import MarkerStyle
import matplotlib.pyplot as plt

Vamos a trabajar con el sistema dado por la siguiente ecuación de transferencia:

$$
\frac{s+1}{s^2+20s+300}
$$

Primero vamos a ver el LGR con la función `root_locus` de la librería `control`

In [ ]:
S = control.TransferFunction([1, 1], [1, 20, 300])
plt.figure(figsize=(15, 10))
_ = control.root_locus(S, xlim=(-20, 0))

Podemos ver que las líneas correspondientes a la coordenada $\theta$ polar están etiquetadas con el valor del amortiguamiento, en este caso vemos que el amortiguamiento del sistema es $\xi\approx 0.6$. Por otro lado el módulo a los polos nos da el valor de la frecuencia natural del sistema, ahora mismo estamos conciderando que estamos en los puntos marcados por `x` azules que es donde $K=1$, por lo tanto el valor de $\omega_n \approx 17.3$

Con estos datos podemos calcular un estimado para $t_s$ que es el tiempo de estabilización. 

$$
t_s \approx \frac{3.2}{\xi \omega_n} \approx 0.31 [\text{s}]
$$

esta aproximación sirve para $\xi \in [0, 0.69]$

Veamos la respuesta del sistema en lazo cerrado con $K=1$ (que es lo mismo que solo ver la respuesta del sistema solo en lazo cerrado)

In [ ]:
t, y = control.step_response(S)
plt.figure(figsize=(15, 5))
plt.xlabel("Tiempo [s]")
plt.ylabel("respuesta al escalón y(t)")
plt.vlines(.31, 0, .03, "r", '--')
plt.plot(t, y)
plt.grid()
plt.text(.32, .028, "Estimación", c="red")
plt.title("Respuesta del sistema en lazo cerrado con $K=1$");

Ahora vamos a probar con $K=10$

In [ ]:
S_ = S * PID(10, 0, 0)
plt.figure(figsize=(15, 10))
_ = control.root_locus(S, xlim=(-20, 0))
plt.scatter(closed_loop(S_).pole().real, closed_loop(S_).pole().imag, c='r', s=100);

Podemos ver que los nuevos polos están donde están los puntos rojos.

Con estos nueos valores podemos nuevamente estimar $t_s$. Tenemos que encontrar $\xi$ y $\omega_n$ ahora tenemos que usar otra aproximación

$$
t_s\approx \frac{4.5 \xi}{\omega_n}
$$

pues $\xi>0.69$

Afortunadamente no hace falta que escribamos funciones para esto pues `control` nos permite hacer esto con `TransferFunction.damp()` 


In [ ]:
w_n, xi, _ = closed_loop(S_).damp()
t_s = 4.5 * xi / w_n
t_s = t_s[0]
t_s

Así obtenemos que $t_s\approx 0.21$. Veamos la respuesta para ver si tiene sentido

In [ ]:
t, y = control.step_response(closed_loop(S_))
plt.figure(figsize=(15, 5))
plt.xlabel("Tiempo [s]")
plt.ylabel("respuesta al escalón y(t)")
plt.vlines(.22, 0, .25, "r", '--')
plt.plot(t, y)
plt.grid()
plt.text(.23, .23, "Estimación", c="red")
plt.title("Respuesta del sistema en lazo cerrado con $K=10$");

Finalmente intentemos aumentar aún más el valor de $K$, con $K=20$

In [ ]:
S_ = S * PID(20, 0, 0)
plt.figure(figsize=(15, 10))
_ = control.root_locus(S, xlim=(-35, 0))
plt.scatter(closed_loop(S_).pole().real, closed_loop(S_).pole().imag, c='r', s=100);

In [ ]:
w_n, xi, _ = closed_loop(S_).damp()
t_s = 4.5 * xi / w_n
t_s = t_s[0]
t_s

In [ ]:
t, y = control.step_response(closed_loop(S_))
plt.figure(figsize=(15, 5))
plt.xlabel("Tiempo [s]")
plt.ylabel("respuesta al escalón y(t)")
plt.vlines(.16, 0, .4, "r", '--')
plt.plot(t, y)
plt.grid()
plt.text(.17, .36, "Estimación", c="red")
plt.title("Respuesta del sistema en lazo cerrado con $K=10$");

Por algún motivo las estimaciones no son muy buenas, por lo que quizá sea mejor usar la siguiente fórmulas:


$$
t_s = -\frac{\log\left(s \sqrt{1-\xi^2}\right)}{\xi \omega_n}
$$

donde $s$ es el valor de estabilización